# Quickstart

Heat a small workshop with a gas boiler — the minimal working example.

This notebook introduces the **core concepts** of fluxopt:

- **Bus**: Balance nodes where energy flows meet
- **Flow**: Energy transfer on a bus — with size, bounds, and cost coefficients
- **Effect**: Quantities to track and optimize (costs, emissions)
- **Port**: Sources and sinks connecting the system to the outside world
- **Converter**: Linear coupling between input and output flows

## Setup

In [ ]:
from datetime import datetime

import plotly.express as px
import plotly.io as pio

from fluxopt import Bus, Converter, Effect, Flow, Port, optimize

pio.renderers.default = 'notebook_connected'

## Define the Time Horizon

Every optimization needs a time horizon. Here we model a simple 4-hour period:

In [ ]:
timesteps = [datetime(2024, 1, 15, h) for h in range(8, 12)]
timesteps

## Define the Heat Demand

The workshop has varying heat demand throughout the morning:

In [ ]:
heat_demand = [30, 50, 45, 25]  # kW per hour

px.bar(x=timesteps, y=heat_demand, labels={'x': 'Time', 'y': 'kW'}, title='Heat Demand')

## Build and Solve the Energy System

The system has a gas grid, a boiler, and a heat demand:

```
  Gas Grid ──► [gas bus] ──► Boiler ──► [heat bus] ──► Workshop
      €                        η=90%                    Demand
```

In fluxopt, we define **Flows** (energy on a bus), group them into **Ports**
(sources/sinks) and **Converters** (coupled flows), then call `optimize`.

In [ ]:
# Gas supply: up to 1000 kW, costs 0.08 EUR/kWh
gas_source = Flow(bus='gas', size=1000, effects_per_flow_hour={'cost': 0.08})

# Boiler flows
fuel = Flow(bus='gas', size=200)
heat_out = Flow(bus='heat', size=100)

# Heat demand: 50 kW capacity * relative profile = [30, 50, 45, 25] kW
demand_flow = Flow(bus='heat', size=50, fixed_relative_profile=[0.6, 1.0, 0.9, 0.5])

result = optimize(
    timesteps=timesteps,
    buses=[Bus('gas'), Bus('heat')],
    effects=[Effect('cost', unit='EUR', is_objective=True)],
    ports=[
        Port('gas_grid', imports=[gas_source]),
        Port('workshop', exports=[demand_flow]),
    ],
    converters=[
        Converter.boiler('boiler', thermal_efficiency=0.9, fuel_flow=fuel, thermal_flow=heat_out),
    ],
)

## Analyze Results

### Total Costs

The total cost — the minimized objective:

In [ ]:
total_heat = sum(heat_demand)
gas_consumed = total_heat / 0.9

print(f'Total heat demand: {total_heat:.0f} kWh')
print(f'Gas consumed:      {gas_consumed:.1f} kWh')
print(f'Total cost:        {result.objective:.2f} EUR')
print(f'Avg cost of heat:  {result.objective / total_heat:.4f} EUR/kWh')

### Flow Rates

Visualize all flow rates over time. Flow ids are qualified as `{component}({bus})`.

In [ ]:
df = result.flow_rates.to_dataframe('kW').reset_index()
px.line(df, x='time', y='kW', color='flow', title='Flow Rates', line_shape='hv')

Use `flow_rate()` to get a single flow's time series:

In [ ]:
df = result.flow_rate('boiler(gas)').to_dataframe('kW').reset_index()
px.line(df, x='time', y='kW', title='Boiler Gas Consumption', line_shape='hv')

### Effect Totals

In [ ]:
result.effect_totals

### Per-Timestep Effects

Cost incurred at each timestep:

In [ ]:
df = result.effects_temporal.to_dataframe('EUR').reset_index()
px.bar(df, x='time', y='EUR', color='effect', title='Effects per Timestep')

## Summary

The basic workflow:

1. **Define** flows with size, bounds, and cost coefficients
2. **Group** flows into ports (sources/sinks) and converters
3. **Optimize** with `optimize(timesteps, buses, effects, ports, converters)`
4. **Inspect** results via `result.flow_rates`, `result.effect_totals`, `result.effects_temporal`

### Next Steps

- [02-heat-system](02-heat-system.ipynb): Add thermal storage to shift loads with time-varying prices